In [1]:
import requests
from bs4 import BeautifulSoup as bs
from parse_methods import parse_method
#from parse_ingredients import parse_ingredient

Take in a URL

In [2]:
url = 'https://www.allrecipes.com/recipe/24074/alysias-basic-meat-lasagna/'
page = requests.get(url)
soup = bs(page.content, "html.parser")

In [7]:
title = soup.find("h1", class_="headline heading-content elementFont__display").text

In [58]:
recipe_info = soup.find_all("div", class_="recipe-meta-item-header elementFont__subtitle--bold elementFont__transformCapitalize")
recipe_info = [x.text for x in recipe_info]
recipe_info_details = soup.find_all("div", class_="recipe-meta-item-body elementFont__subtitle")
recipe_info_details = [x.text[:-1] for x in recipe_info_details]

recipe_info = zip(recipe_info, recipe_info_details)
for item in recipe_info:
  print(item)

('prep:', '50 mins')
('cook:', '45 mins')
('total:', '1 hr 35 mins')
('Servings:', '10')
('Yield:', '10 servings')


In [74]:
ingredients = soup.find_all("span", class_="ingredients-item-name elementFont__body")
# ingredients = [x.text for x in ingredients]
# need to translate some encodings and special characters
# also need to learn how to distinguish ingredients and measurements

In [49]:
# Source: https://stackoverflow.com/questions/15190930/python-beautifulsoup-parsing-html-fractions

import unicodedata as ud
import re

numerator = {
    'ONE':1,
    'TWO':2,
    'THREE':3,
    'FOUR':4,
    'FIVE':5,
    'SIX':6,
    'SEVEN':7,
    'EIGHT':8,
    'NINE':9,
    'ZERO':0,
    }

denominator = {
    'QUARTER':4,
    'HALF':2,
    'SEVENTH':7,
    'NINTH':9,
    'THIRD':3,
    'FIFTH':5,
    'SIXTH':6,
    'EIGHTH':8,
    'SIXTEENTH':16
    }

fraction = {}
for num in range(0x110000):
    s = chr(num)
    try:
        name = ud.name(s)
    except ValueError:
        continue
    match = re.search('FRACTION ({n}) ({d})'.format(
        n = '|'.join(numerator.keys()),
        d = '|'.join(denominator.keys()),
        ) , name)
    if match:
        fraction[num] = str(
           float(numerator[match.group(1)])/denominator[match.group(2)]).lstrip('0')
print(fraction)

{188: '.25', 189: '.5', 190: '.75', 2930: '.25', 2931: '.5', 2932: '.75', 2933: '.0625', 2934: '.125', 2935: '.1875', 3422: '.2', 3443: '.25', 3444: '.5', 3445: '.75', 3446: '.0625', 3447: '.125', 3448: '.1875', 8528: '.14285714285714285', 8529: '.1111111111111111', 8531: '.3333333333333333', 8532: '.6666666666666666', 8533: '.2', 8534: '.4', 8535: '.6', 8536: '.8', 8537: '.16666666666666666', 8538: '.8333333333333334', 8539: '.125', 8540: '.375', 8541: '.625', 8542: '.875', 8585: '.0', 11517: '.5', 43056: '.25', 43057: '.5', 43058: '.75', 43059: '.0625', 43060: '.125', 43061: '.1875', 68029: '.5', 68168: '.5', 69243: '.5', 69244: '.25', 69245: '.3333333333333333', 69246: '.6666666666666666', 69414: '.5', 73673: '.0625', 73674: '.0625', 73676: '.125', 73678: '.1875', 73679: '.2', 73680: '.25', 73681: '.5', 73682: '.5', 73683: '.75', 126125: '.25', 126126: '.5', 126127: '.75', 126268: '.5', 126269: '.16666666666666666'}


In [75]:
ingredients = [ingredient.text.strip().translate(fraction).encode('ascii', 'ignore').decode("utf-8") for ingredient in ingredients]
print(ingredients)

['1.5 pounds ground beef', '1 teaspoon garlic powder', '1 (28 ounce) jar sausage flavored spaghetti sauce', '1 (8 ounce) can tomato sauce', '1 teaspoon dried oregano', '1 tablespoon olive oil', '4 cloves garlic, minced', '1 small onion, diced', '1 (8 ounce) package mozzarella cheese, shredded', '8 ounces provolone cheese, shredded', '1 (15 ounce) container ricotta cheese', '2 eggs', '.25 cup milk', '.5 teaspoon dried oregano', '9 lasagna noodles', '.25 cup grated Parmesan cheese']


In [76]:
parsed_ingredients = []
for ingredient in ingredients:
    print(parse_ingredient(ingredient))
    parsed_ingredients.append(parse_ingredient(ingredient))

Ingredient(name='ground beef', quantity=1.5, unit='lb', comment='', original_string='1.5 pounds ground beef')
Ingredient(name='garlic powder', quantity=1.0, unit='tsp', comment='', original_string='1 teaspoon garlic powder')
Ingredient(name='jar sausage flavored spaghetti sauce', quantity=1.0, unit='', comment='(28 ounce)', original_string='1 (28 ounce) jar sausage flavored spaghetti sauce')
Ingredient(name='can tomato sauce', quantity=1.0, unit='', comment='(8 ounce)', original_string='1 (8 ounce) can tomato sauce')
Ingredient(name='dried oregano', quantity=1.0, unit='tsp', comment='', original_string='1 teaspoon dried oregano')
Ingredient(name='olive oil', quantity=1.0, unit='tbsp', comment='', original_string='1 tablespoon olive oil')
Doing split for 4 cloves garlic, minced
Ingredient(name='garlic', quantity=4.0, unit='clove', comment='minced', original_string='4 cloves garlic, minced')
Doing split for 4 cloves garlic, minced
Doing split for 1 small onion, diced
Ingredient(name='sma

In [3]:
steps = soup.find_all("div", class_="section-body elementFont__body--paragraphWithin elementFont__body--linkWithin")
steps = [x.text[:-2] for x in steps]

In [11]:
nutrition = soup.find("div", class_="recipeNutritionSectionBlock")
nutrition = nutrition.text[1:-17]

nutrition

'Per Serving:  567 calories; protein 36g; carbohydrates 35.6g; fat 31.4g; cholesterol 129.9mg; sodium 943.1mg.'

Methods

In [4]:
methods = soup.find_all("div", class_="paragraph")
directions = [method.text.strip().encode('ascii', 'ignore').decode("utf-8") for method in methods]
print(directions)

['Preheat oven to 375 degrees F (190 degrees C).', 'Season ground beef with garlic powder. Heat a large skillet over medium-high heat. Cook and stir ground beef in the hot skillet until browned and crumbly, 5 to 7 minutes. Drain and discard grease.', 'Pour spaghetti sauce, tomato sauce, and oregano into a large saucepan. Set aside.', 'Heat olive oil in a large skillet over medium-high heat. Saute garlic and onions until softened and translucent, about 5 minutes. Place cooked onion-garlic mixture and cooked ground beef into the sauce mixture. Cover and let simmer for 15 to 20 minutes.', 'Combine mozzarella and provolone cheeses in a medium bowl. Mix ricotta cheese, eggs, milk, and 1/2 teaspoon oregano together in a separate bowl.', 'Layer a 9x13-inch baking pan with just enough sauce to cover the bottom of the pan. Lay three lasagna noodles in the pan over the sauce. Cover with more sauce, then with ricotta mixture then sprinkle with mozzarella/provolone mixture; repeat layering. Finish

In [6]:
methods = []
for i in range(len(directions)):
    method = parse_method(directions[i], i)
    methods.append(method)
    print(method.direction, method.primary_cooking, method.secondary_cooking)

Preheat oven to 375 degrees F (190 degrees C). ['preheat'] []
Season ground beef with garlic powder. Heat a large skillet over medium-high heat. Cook and stir ground beef in the hot skillet until browned and crumbly, 5 to 7 minutes. Drain and discard grease. ['brown'] ['heat', 'cook', 'stir']
Pour spaghetti sauce, tomato sauce, and oregano into a large saucepan. Set aside. ['set'] []
Heat olive oil in a large skillet over medium-high heat. Saute garlic and onions until softened and translucent, about 5 minutes. Place cooked onion-garlic mixture and cooked ground beef into the sauce mixture. Cover and let simmer for 15 to 20 minutes. ['saute'] ['soften', 'cook', 'cook', 'cover', 'let']
Combine mozzarella and provolone cheeses in a medium bowl. Mix ricotta cheese, eggs, milk, and 1/2 teaspoon oregano together in a separate bowl. ['mix'] ['combine', 'cheese']
Layer a 9x13-inch baking pan with just enough sauce to cover the bottom of the pan. Lay three lasagna noodles in the pan over the s